In [ ]:
# default_exp core

# Core

> Functions to setup a colab runtime for use with nbdev and github.

The goal of this project is to make it as easy as possible to work with nbdev and github without leaving colab.

## Regular nbdev notebooks

As you can see from this notebook, you don't have to `!pip install nbdev` in every notebook.

You can use the following pattern to avoid running colab specific things in non-colab environments, like mounting your google drive:

```python
IN_COLAB = 'google.colab' in str(get_ipython())
if IN_COLAB:
  from google.colab import drive
```

or installing nbdev:

```python
if IN_COLAB:
  !pip install git+https://github.com/fastai/nbdev.git
```

Now that nbdev is available to the runtime, you can `from nbdev import *` and start using [magic flags](https://pete88b.github.io/fastpages/nbdev/fastai/jupyter/2020/06/02/nbdev-magic.html), `show_doc` etc.

## The colab helper notebook

Having a colab helper notebook gives you a place to do the things you would do from the command line in a non-colab environment (like running nbdev or git commands). Feel free to use [this notebook](https://github.com/pete88b/nbdev_colab_helper/blob/master/_colab_helper.ipynb) as a starter.

Before we can run nbdev or git commands in a colab helper notebook, we need to set-up the colab runtime via a call to `nbdev_colab_helper.core.setup_project`.

After mounting your google drive to `/content/drive`, `setup_project` looks for `/content/drive/My Drive/nbdev_colab_projects.ini`: which must have a section for the project you are setting up.

**Warning:** Your nbdev_colab_projects.ini should live outside your nbdev projects. Please don't share this file, or push to githib etc

Here's an example `nbdev_colab_projects.ini` with sections for 2 projects:

```
[DEFAULT]
project_parent = /content/drive/My Drive/Colab Notebooks/github

git_user_name = pete88b
git_user_email = **secret**@gmail.com
git_user_password = **secret**

[nbdev_colab_helper]
git_url = https://github.com/pete88b/nbdev_colab_helper.git
git_branch = master

[nbdev_demo]
git_url = https://github.com/pete88b/nbdev_demo.git
git_branch = main
```

See [`ConfigParser`](https://docs.python.org/3/library/configparser.html) for details on how this file can be formatted.

The name of the colab helper notebook is not important and it doesn't have to be part of your nbdev project - you could manage any number of nbdev projects from a single colab helper notebook.

In [ ]:
#exports
IN_COLAB = 'google.colab' in str(get_ipython())

In [ ]:
#export
import os, subprocess, urllib, shlex

def _run_commands(commands, password=None):
  "Run a list of commands making sure we mask `password` when printing commands"
  for cmd in commands:
    process = subprocess.Popen(shlex.split(cmd), stdout=subprocess.PIPE)
    output, err = process.communicate()
    if password: cmd = cmd.replace(password, '*****')
    print(cmd)
    if output or err:
      print('  ', output.decode('utf8').strip() if output else '', err or '')

def setup_git(git_url:str, git_branch:str, name:str, password:str, email:str):
  "Link your mounted drive to GitHub"
  password = urllib.parse.quote(password)
  _run_commands([
      f"git config --global user.email {email}",
      f"git config --global user.name {name}",
      f"git init",
      f"git remote rm origin",
      f"git remote add origin {git_url.replace('://git', f'://{name}:{password}@git')}",
      f"git pull origin {git_branch} --allow-unrelated-histories"],
      password)

def git_push(git_branch:str, message:str):
  "Convert the notebooks to scripts and then push to the library"
  _run_commands([
      f'nbdev_install_git_hooks',
      f'nbdev_build_lib',
      f'git add *', # TODO: allow specific files to be pushed
      f'git commit -m "{message}"',
      f'git push origin {git_branch}'])

In [ ]:
#export
if IN_COLAB:
  from google.colab import drive
from configparser import ConfigParser
from pathlib import Path

def setup_project(project_name):
  "Set-up the colab runtime for `project_name`"
  print('Connecting to google drive')
  drive.mount('/content/drive')
  config_path = Path('/content/drive/My Drive/nbdev_colab_projects.ini')
  config = ConfigParser()
  config.read(config_path)
  if project_name not in config:
    print(f'Error: [{project_name}] section not found in {config_path}')
    print(f'Please add a section for [{project_name}] and run `setup_project` again')
    print('See https://pete88b.github.io/nbdev_colab_helper/core.html for details')
    return
  project_config = config[project_name]
  project_path = Path(project_config['project_parent'])/project_name
  git_url, git_branch = project_config['git_url'], project_config['git_branch']
  if project_path.is_dir():
    print(f'Clone of {project_name} already exists in {project_path.parent}')
  else:
    project_path.parent.mkdir(parents=True, exist_ok=True)
    get_ipython().magic(f'cd {project_path.parent}')
    _run_commands([f'git clone {git_url}'])
  get_ipython().magic(f'cd {project_path}')
  _run_commands(['pip install git+https://github.com/fastai/nbdev.git'])
  setup_git(git_url, git_branch, project_config['git_user_name'], 
            project_config['git_user_password'], project_config['git_user_email'])
  return config, project_config